IMPORTAZIONE LIBRERIE

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as signal
from scipy.fft import fft, fftfreq
from scipy.signal import find_peaks

CAMBIO PATH PER ANALISI RISPETTO AL LOG DI VOLO CONSIDERATO

In [2]:
path_file = os.path.abspath(os.getcwd())
print(path_file)
os.chdir(path_file)

c:\Users\chiar\OneDrive\Desktop\MantenzionePreventiva\ProgettoManutenzionePreventiva\M1\NO_FAULT1


VARIABILE RCOU POTENZA MOTORI

In [3]:
#andiamo a prendere una potenza media di 1400 per considerare il momento il cui inizia/finisce il volo
rcou_m1 = pd.read_csv("RCOU.csv")
rcou_m1 = rcou_m1.drop(['LineNo','C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8'], axis=1)

rcou_m1 = rcou_m1[((rcou_m1['C9'] >= 1350) & (rcou_m1['C10'] >= 1350) & (rcou_m1['C11'] >= 1350) & (rcou_m1['C12'] >= 1350) & (rcou_m1['C13'] >= 1350) & (rcou_m1['C14']>= 1350))]

pd.set_option('display.max_rows',700)
display(rcou_m1)

,TimeUS,C9,C10,C11,C12,C13,C14
710,22664152,1360,1375,1382,1352,1371,1363
711,22676651,1382,1369,1360,1390,1364,1387
720,22771304,1452,1411,1456,1406,1473,1386
721,22782035,1507,1358,1409,1465,1479,1393
722,22792632,1519,1355,1384,1497,1465,1420
...,...,...,...,...,...,...,...
4966,68488846,1553,1460,1514,1503,1422,1584
4967,68499144,1536,1469,1506,1501,1397,1594
4968,68509358,1542,1448,1511,1482,1419,1565
4969,68523070,1572,1386,1463,1509,1415,1550


In [4]:
#xkf utilizzato per fare poi il confronto con i valori di roll, pitch e yaw in att
xkf1_m1 = pd.read_csv("XKF1_0.csv")
xkf1_m1 = xkf1_m1.drop(['LineNo','C','VN','VE','VD','dPD','PN','PE','PD','GX','GY','GZ','OH'], axis=1)
xkf1_m1 = xkf1_m1[((xkf1_m1['TimeUS'] >= 22664152) & (xkf1_m1['TimeUS'] <= 68533079))]
print(xkf1_m1)

        TimeUS  Roll  Pitch    Yaw
712   22675051  0.01  -0.44  13.24
713   22685153  0.03  -0.40  13.24
714   22695762 -0.03  -0.45  13.23
715   22705793 -0.04  -0.52  13.22
716   22715779  0.03  -0.40  13.23
...        ...   ...    ...    ...
4968  68486359 -4.46   1.19   5.63
4969  68496743 -4.40   1.35   5.58
4970  68507280 -4.34   1.52   5.53
4971  68521132 -4.14   1.73   5.49
4972  68531134 -3.95   1.91   5.45

[4261 rows x 4 columns]


In [5]:
att_m1 = pd.read_csv("ATT.csv")
att_m1 = att_m1.drop(['LineNo','ErrRP', 'ErrYaw', 'AEKF'], axis=1)
att_m1 = att_m1[((att_m1['TimeUS'] >= 22664152) & (att_m1['TimeUS'] <= 68533079))]
print(att_m1)

        TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw    Yaw
712   22675940     0.00  0.01      0.00  -0.44   13.23  13.24
713   22686733     0.00  0.03      0.00  -0.40   13.23  13.24
714   22696746     0.00 -0.03      0.00  -0.45   13.23  13.23
715   22706870     0.00 -0.04      0.00  -0.52   13.23  13.22
716   22716835     0.00  0.03      0.00  -0.40   13.23  13.23
...        ...      ...   ...       ...    ...     ...    ...
4968  68488238    -1.70 -4.46     -0.22   1.19    5.99   5.63
4969  68498617    -1.59 -4.40     -0.21   1.35    5.99   5.58
4970  68508630    -1.49 -4.34     -0.20   1.52    5.99   5.53
4971  68522298    -1.39 -4.14     -0.18   1.73    5.99   5.49
4972  68532571    -1.30 -3.95     -0.17   1.91    5.99   5.45

[4261 rows x 7 columns]


In [6]:
#controllo corrispondenza valori roll, pitch, yaw
att_m1 = pd.merge_asof(att_m1, xkf1_m1, on="TimeUS", by=["Roll","Pitch","Yaw"], tolerance=100000)
print(att_m1)

        TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw    Yaw
0     22675940     0.00  0.01      0.00  -0.44   13.23  13.24
1     22686733     0.00  0.03      0.00  -0.40   13.23  13.24
2     22696746     0.00 -0.03      0.00  -0.45   13.23  13.23
3     22706870     0.00 -0.04      0.00  -0.52   13.23  13.22
4     22716835     0.00  0.03      0.00  -0.40   13.23  13.23
...        ...      ...   ...       ...    ...     ...    ...
4256  68488238    -1.70 -4.46     -0.22   1.19    5.99   5.63
4257  68498617    -1.59 -4.40     -0.21   1.35    5.99   5.58
4258  68508630    -1.49 -4.34     -0.20   1.52    5.99   5.53
4259  68522298    -1.39 -4.14     -0.18   1.73    5.99   5.49
4260  68532571    -1.30 -3.95     -0.17   1.91    5.99   5.45

[4261 rows x 7 columns]


In [7]:
#elettricita' che scorre nel motore considerato, al momento ne consideriamo solo 1
esc_0_m1 = pd.read_csv("ESC_0.csv")
esc_0_m1 = esc_0_m1.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_0_m1 = esc_0_m1[((esc_0_m1['TimeUS'] >= 22664152) & (esc_0_m1['TimeUS'] <= 68533079))]
esc_0_m1=esc_0_m1.rename(columns={"RPM": "RPM(ESC_0)", "Curr": "Curr(ESC_0)"})
print(esc_0_m1)
esc_1_m1 = pd.read_csv("ESC_1.csv")
esc_1_m1 = esc_1_m1.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_1_m1 = esc_1_m1[((esc_1_m1['TimeUS'] >= 22664152) & (esc_1_m1['TimeUS'] <= 68533079))]
esc_1_m1=esc_1_m1.rename(columns={"RPM": "RPM(ESC_1)", "Curr": "Curr(ESC_1)"})

esc_2_m1 = pd.read_csv("ESC_2.csv")
esc_2_m1 = esc_2_m1.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_2_m1 = esc_2_m1[((esc_2_m1['TimeUS'] >= 22664152) & (esc_2_m1['TimeUS'] <= 68533079))]
esc_2_m1=esc_2_m1.rename(columns={"RPM": "RPM(ESC_2)", "Curr": "Curr(ESC_2)"})

esc_3_m1 = pd.read_csv("ESC_3.csv")
esc_3_m1 = esc_3_m1.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_3_m1 = esc_3_m1[((esc_3_m1['TimeUS'] >= 22664152) & (esc_3_m1['TimeUS'] <= 68533079))]
esc_3_m1=esc_3_m1.rename(columns={"RPM": "RPM(ESC_3)", "Curr": "Curr(ESC_3)"})

esc_4_m1 = pd.read_csv("ESC_4.csv")
esc_4_m1 = esc_4_m1.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_4_m1 = esc_4_m1[((esc_4_m1['TimeUS'] >= 22664152) & (esc_4_m1['TimeUS'] <= 68533079))]
esc_4_m1=esc_4_m1.rename(columns={"RPM": "RPM(ESC_4)", "Curr": "Curr(ESC_4)"})

esc_5_m1 = pd.read_csv("ESC_5.csv")
esc_5_m1 = esc_5_m1.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_5_m1 = esc_5_m1[((esc_5_m1['TimeUS'] >= 22664152) & (esc_5_m1['TimeUS'] <= 68533079))]
esc_5_m1=esc_5_m1.rename(columns={"RPM": "RPM(ESC_5)", "Curr": "Curr(ESC_5)"})



       TimeUS  RPM(ESC_0)  Curr(ESC_0)
68   22742780        3495         2.16
69   22845457        4097         2.65
70   22959904        4696         3.34
71   23059012        5180         4.33
72   23167332        5753         4.29
73   23272089        6060         3.92
74   23387634        6310         4.54
75   23499927        6266         4.03
76   23602066        6227         3.98
77   23709666        6214         4.15
78   23821462        6291         4.02
79   23922931        6084         4.91
80   24140073        6394         5.41
81   24241826        6617         4.74
82   24355612        6651         4.40
83   24458193        6428         4.01
84   24563045        6225         3.88
85   24677142        6148         4.24
86   24780143        6059         3.99
87   24894895        6102         3.90
88   25000816        5953         4.13
89   25111147        6179         3.85
90   25216093        6107         3.97
91   25322971        6011         3.83
92   25431084        5965

In [8]:
#valori di accelerazione e del giroscopio
imu_2_m1 = pd.read_csv("IMU_2.csv")
imu_2_m1 = imu_2_m1.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_2_m1 = imu_2_m1[((imu_2_m1['TimeUS'] >= 22664152) & (imu_2_m1['TimeUS'] <= 68533079))]
print(imu_2_m1)

         TimeUS      GyrX      GyrY      GyrZ      AccX      AccY       AccZ
2772   22666080  0.003473 -0.071788 -0.003969 -0.288769  0.070756  -9.615868
2773   22668252  0.022022 -0.018390 -0.004186 -0.273620  0.065257  -9.677919
2774   22670635  0.042835  0.047712  0.003529 -0.285327  0.087617  -9.837013
2775   22675772  0.041332  0.027353 -0.005676 -0.265016  0.116323  -9.899507
2776   22681012 -0.047680 -0.088542 -0.020646 -0.149086  0.106717  -9.691211
...         ...       ...       ...       ...       ...       ...        ...
19275  68522130  0.284771  0.282203 -0.021080 -0.704096  2.665057 -11.796820
19276  68524886  0.340421  0.269412 -0.034364 -0.749058  2.835196 -12.038420
19277  68527023  0.308373  0.309697 -0.031700 -0.718123  2.934844 -12.225820
19278  68530131  0.320004  0.317270 -0.023956 -0.648460  3.013909 -12.377950
19279  68532323  0.333978  0.296047 -0.020278 -0.633987  3.028393 -12.458560

[16508 rows x 7 columns]


In [9]:
#sincronizzazione dei tempi tra att e imu_2
m1_nofault = pd.merge_ordered(imu_2_m1,att_m1)
m1_nofault=m1_nofault.fillna(method="ffill").fillna(method="bfill")
print(m1_nofault)

         TimeUS      GyrX      GyrY      GyrZ      AccX      AccY       AccZ  \
0      22666080  0.003473 -0.071788 -0.003969 -0.288769  0.070756  -9.615868   
1      22668252  0.022022 -0.018390 -0.004186 -0.273620  0.065257  -9.677919   
2      22670635  0.042835  0.047712  0.003529 -0.285327  0.087617  -9.837013   
3      22675772  0.041332  0.027353 -0.005676 -0.265016  0.116323  -9.899507   
4      22675940  0.041332  0.027353 -0.005676 -0.265016  0.116323  -9.899507   
...         ...       ...       ...       ...       ...       ...        ...   
20764  68524886  0.340421  0.269412 -0.034364 -0.749058  2.835196 -12.038420   
20765  68527023  0.308373  0.309697 -0.031700 -0.718123  2.934844 -12.225820   
20766  68530131  0.320004  0.317270 -0.023956 -0.648460  3.013909 -12.377950   
20767  68532323  0.333978  0.296047 -0.020278 -0.633987  3.028393 -12.458560   
20768  68532571  0.333978  0.296047 -0.020278 -0.633987  3.028393 -12.458560   

       DesRoll  Roll  DesPitch  Pitch  

In [10]:
#sincronizzazione dei tempi tra la tabella ottenuta precedentemente e esc_0 (corrente che passa nel motore)
m1_nofault = pd.merge_ordered(m1_nofault, esc_0_m1)
m1_nofault = pd.merge_ordered(m1_nofault, esc_1_m1)
tabFault = pd.merge_ordered(m1_nofault, esc_2_m1)
m1_nofault = pd.merge_ordered(m1_nofault, esc_3_m1)
m1_nofault = pd.merge_ordered(m1_nofault, esc_4_m1)
m1_nofault = pd.merge_ordered(m1_nofault, esc_5_m1)
m1_nofault=m1_nofault.fillna(method="ffill").fillna(method="bfill")
m1_nofault = pd.merge_ordered(m1_nofault, rcou_m1)
m1_nofault=m1_nofault.fillna(method="ffill").fillna(method="bfill")

print(m1_nofault)

         TimeUS      GyrX      GyrY      GyrZ      AccX      AccY       AccZ  \
0      22664152  0.003473 -0.071788 -0.003969 -0.288769  0.070756  -9.615868   
1      22666080  0.003473 -0.071788 -0.003969 -0.288769  0.070756  -9.615868   
2      22668252  0.022022 -0.018390 -0.004186 -0.273620  0.065257  -9.677919   
3      22670635  0.042835  0.047712  0.003529 -0.285327  0.087617  -9.837013   
4      22675772  0.041332  0.027353 -0.005676 -0.265016  0.116323  -9.899507   
...         ...       ...       ...       ...       ...       ...        ...   
26995  68527023  0.308373  0.309697 -0.031700 -0.718123  2.934844 -12.225820   
26996  68530131  0.320004  0.317270 -0.023956 -0.648460  3.013909 -12.377950   
26997  68532323  0.333978  0.296047 -0.020278 -0.633987  3.028393 -12.458560   
26998  68532571  0.333978  0.296047 -0.020278 -0.633987  3.028393 -12.458560   
26999  68533079  0.333978  0.296047 -0.020278 -0.633987  3.028393 -12.458560   

       DesRoll  Roll  DesPitch  ...  RP

In [11]:
index_to_remove=[]
for ind in att_m1["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_0_m1["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_1_m1["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_2_m1["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_3_m1["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_4_m1["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_5_m1["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in rcou_m1["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
print("--------------------")
print(len(index_to_remove))

4261
4671
5082
5492
5900
6305
6717
10902
--------------------
10902


In [12]:
#aggiornamento del tempo ponendo a 0 il momento in cui il volo inizia per noi e settaggio della variabile guasto
#from datetime import timedelta
#index_to_remove=set(set(set(set(set(set(set(att_m1["TimeUS"]).add(esc_0_m1.index)).add(esc_1_m1.index)).add(esc_2_m1.index)).add(esc_3_m1.index)).add(esc_4_m1.index)).add(esc_5_m1.index)).add(rcou_m1.index)
#m1_nofault.drop(index_to_remove, axis=0, inplace=True)
m1_nofault = m1_nofault[~m1_nofault.TimeUS.isin(index_to_remove)]
m1_nofault["TimeUS"] = m1_nofault["TimeUS"] - m1_nofault.iloc[0]["TimeUS"]
m1_nofault["TimeUS"] = m1_nofault["TimeUS"].astype(int)
m1_nofault["Guasto"] = 0
print(m1_nofault)

         TimeUS      GyrX      GyrY      GyrZ      AccX      AccY       AccZ  \
1             0  0.003473 -0.071788 -0.003969 -0.288769  0.070756  -9.615868   
2          2172  0.022022 -0.018390 -0.004186 -0.273620  0.065257  -9.677919   
3          4555  0.042835  0.047712  0.003529 -0.285327  0.087617  -9.837013   
4          9692  0.041332  0.027353 -0.005676 -0.265016  0.116323  -9.899507   
7         14932 -0.047680 -0.088542 -0.020646 -0.149086  0.106717  -9.691211   
...         ...       ...       ...       ...       ...       ...        ...   
26991  45856050  0.284771  0.282203 -0.021080 -0.704096  2.665057 -11.796820   
26994  45858806  0.340421  0.269412 -0.034364 -0.749058  2.835196 -12.038420   
26995  45860943  0.308373  0.309697 -0.031700 -0.718123  2.934844 -12.225820   
26996  45864051  0.320004  0.317270 -0.023956 -0.648460  3.013909 -12.377950   
26997  45866243  0.333978  0.296047 -0.020278 -0.633987  3.028393 -12.458560   

       DesRoll  Roll  DesPitch  ...  Cu

In [13]:
from datetime import timedelta
m1_nofault=m1_nofault.reset_index(drop=True)
m1_nofault['TimeUS'] = pd.to_datetime(m1_nofault['TimeUS'], unit='us').dt.strftime('%H:%M:%S.%f')
display(m1_nofault)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ,DesRoll,Roll,DesPitch,...,Curr(ESC_4),RPM(ESC_5),Curr(ESC_5),C9,C10,C11,C12,C13,C14,Guasto
0,00:00:00.000000,0.003473,-0.071788,-0.003969,-0.288769,0.070756,-9.615868,0.00,0.01,0.00,...,1.90,3348.0,1.99,1360.0,1375.0,1382.0,1352.0,1371.0,1363.0,0
1,00:00:00.002172,0.022022,-0.018390,-0.004186,-0.273620,0.065257,-9.677919,0.00,0.01,0.00,...,1.90,3348.0,1.99,1360.0,1375.0,1382.0,1352.0,1371.0,1363.0,0
2,00:00:00.004555,0.042835,0.047712,0.003529,-0.285327,0.087617,-9.837013,0.00,0.01,0.00,...,1.90,3348.0,1.99,1360.0,1375.0,1382.0,1352.0,1371.0,1363.0,0
3,00:00:00.009692,0.041332,0.027353,-0.005676,-0.265016,0.116323,-9.899507,0.00,0.01,0.00,...,1.90,3348.0,1.99,1360.0,1375.0,1382.0,1352.0,1371.0,1363.0,0
4,00:00:00.014932,-0.047680,-0.088542,-0.020646,-0.149086,0.106717,-9.691211,0.00,0.01,0.00,...,1.90,3348.0,1.99,1382.0,1369.0,1360.0,1390.0,1364.0,1387.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16503,00:00:45.856050,0.284771,0.282203,-0.021080,-0.704096,2.665057,-11.796820,-1.49,-4.34,-0.20,...,1.42,5687.0,3.29,1542.0,1448.0,1511.0,1482.0,1419.0,1565.0,0
16504,00:00:45.858806,0.340421,0.269412,-0.034364,-0.749058,2.835196,-12.038420,-1.39,-4.14,-0.18,...,1.42,5687.0,3.29,1572.0,1386.0,1463.0,1509.0,1415.0,1550.0,0
16505,00:00:45.860943,0.308373,0.309697,-0.031700,-0.718123,2.934844,-12.225820,-1.39,-4.14,-0.18,...,1.42,5687.0,3.29,1572.0,1386.0,1463.0,1509.0,1415.0,1550.0,0
16506,00:00:45.864051,0.320004,0.317270,-0.023956,-0.648460,3.013909,-12.377950,-1.39,-4.14,-0.18,...,1.42,5687.0,3.29,1572.0,1386.0,1463.0,1509.0,1415.0,1550.0,0


In [14]:
path_file = path_file.replace(r"M1\NO_FAULT1", "")
os.chdir(path_file)
m1_nofault.to_csv('m1_nofault.csv', index=False)
print(path_file)

c:\Users\chiar\OneDrive\Desktop\MantenzionePreventiva\ProgettoManutenzionePreventiva\


## Analisi nel tempo e in frequenza

In [15]:
import math
#Function that Calculate Root Mean Square
def rmsValue(arr, n):
    square = 0
    mean = 0.0
    root = 0.0
     
    #Calculate square
    for i in range(0,n):
        square += (arr[i]**2)
     
    #Calculate Mean
    mean = (square / (float)(n))
     
    #Calculate Root
    root = math.sqrt(mean)
     
    return root

## Funzione generica per ottenere la tabellina con le feauture nel tempo e in frequenza


In [22]:
def time_freq_feat(V):
    N=350
    height_threshold=0.0
    tab= pd.DataFrame([], columns=[f'Ts({V})',f'<>({V})',f'var({V})',f'kurt({V})',f'RMS({V})',f'freq1({V})',f'freq2({V})',f'amp1({V})',f'amp2({V})'])
    for i in range(int(len(m1_nofault)/350)):
        V1=m1_nofault[str(V)].iloc[:(i+1*350)+1]
        mean=V1.mean()
        var=V1.var()
        kurt=V1.kurtosis()
        rms=rmsValue(V1,len(V1))
        yf=fft(V1.values)
        xf=fftfreq(N,1/350)
        peaks_index, properties = find_peaks(yf,height=-200,prominence=0.1)
        
        #for k in range(len(peaks_index)):
        #[print("%4.4f    \t %3.4f" %(freq[peaks_index[i]], properties['peak_heights'][i])) for i in range(len(peaks_index))]
        h_max_peak_1=np.unique(properties['peak_heights'])[-1]
        peak1_dimension = np.where(properties['peak_heights'] == h_max_peak_1)
        if (len(peak1_dimension[0])>1):
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][1]
        else:
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][0]

        h_max_peak_2=np.unique(properties['peak_heights'])[-2] 
        peak2_dimension = np.where(properties['peak_heights'] == h_max_peak_2)

        if (len(peak2_dimension[0])>1):
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][1]
        else:
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][0]

        val={f'Ts({V})':i,f'<>({V})':mean,f'var({V})':var,f'kurt({V})':kurt,f'RMS({V})':rms,f'freq1({V})':xf[index1],f'freq2({V})':xf[index2],f'amp1({V})':h_max_peak_1,f'amp2({V})':h_max_peak_2}
        tab=tab.append(val,ignore_index=True)
    display(tab)

In [23]:
Yaw=time_freq_feat('Yaw')

c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\chiar\AppData\Local\Temp\ipykernel_32680\342596788.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\chiar\AppData\Local\Temp\ipykernel_32680\342596788.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards th

,Ts(Yaw),<>(Yaw),var(Yaw),kurt(Yaw),RMS(Yaw),freq1(Yaw),freq2(Yaw),amp1(Yaw),amp2(Yaw)
0,0.0,14.672650,2.805611,-0.645370,14.767676,89.0,68.0,-0.798929,-0.966821
1,1.0,14.683466,2.838799,-0.654708,14.779543,92.0,69.0,-0.800948,-1.141577
2,2.0,14.694504,2.873746,-0.661125,14.791689,87.0,65.0,-0.726621,-1.044702
3,3.0,14.705480,2.908252,-0.668622,14.803756,88.0,87.0,-0.676866,-1.036558
4,4.0,14.716394,2.942324,-0.677070,14.815745,95.0,94.0,-0.652218,-0.950345
5,5.0,14.727556,2.978388,-0.682297,14.828045,96.0,95.0,-0.543669,-0.793296
6,6.0,14.738655,3.014002,-0.688657,14.840267,93.0,92.0,-0.462642,-0.677995
7,7.0,14.749693,3.049172,-0.696019,14.852410,92.0,91.0,-0.410133,-0.606029
8,8.0,14.760669,3.083902,-0.704266,14.864476,91.0,90.0,-0.386701,-0.577993
9,9.0,14.771889,3.120635,-0.709459,14.876850,96.0,95.0,-0.283300,-0.484864


In [24]:
AccZ=time_freq_feat('AccZ')

c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\chiar\AppData\Local\Temp\ipykernel_32680\342596788.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\chiar\AppData\Local\Temp\ipykernel_32680\342596788.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards th

,Ts(AccZ),<>(AccZ),var(AccZ),kurt(AccZ),RMS(AccZ),freq1(AccZ),freq2(AccZ),amp1(AccZ),amp2(AccZ)
0,0.0,-10.773729,0.710845,-1.155841,10.806574,0.0,102.0,173.562961,14.754008
1,1.0,-10.772419,0.709423,-1.150419,10.805204,0.0,113.0,174.477052,14.804483
2,2.0,-10.770729,0.708416,-1.145772,10.803472,0.0,104.0,175.509100,14.987545
3,3.0,-10.769139,0.707304,-1.140891,10.801836,0.0,117.0,176.490162,15.133995
4,4.0,-10.767765,0.705976,-1.135557,10.800406,0.0,114.0,177.379026,15.202497
5,5.0,-10.766532,0.704529,-1.129968,10.799109,0.0,123.0,178.202140,15.219519
6,6.0,-10.765477,0.702947,-1.124117,10.797985,0.0,112.0,178.945339,15.171050
7,7.0,-10.764991,0.701063,-1.118060,10.797413,0.0,115.0,179.468779,14.917710
8,8.0,-10.764444,0.699212,-1.111968,10.796783,0.0,114.0,179.996944,14.684646
9,9.0,-10.763206,0.697816,-1.106377,10.795484,0.0,113.0,180.757387,14.699714


In [26]:
GyrX=time_freq_feat('GyrX')
GyrY=time_freq_feat('GyrY')
GyrZ=time_freq_feat('GyrZ')
AccX=time_freq_feat('AccX')	
AccY=time_freq_feat('AccY')
AccZ=time_freq_feat('AccZ')
Roll=time_freq_feat('Roll')
Pitch=time_freq_feat('Pitch')
Yaw=time_freq_feat('Yaw')
RPM_ESC_0=time_freq_feat('RPM(ESC_0)')
CURR_ESC_0=time_freq_feat('Curr(ESC_0)')
RPM_ESC_1=time_freq_feat('RPM(ESC_1)')
CURR_ESC_1=time_freq_feat('Curr(ESC_1)')
RPM_ESC_2=time_freq_feat('RPM(ESC_2)')
CURR_ESC_2=time_freq_feat('Curr(ESC_2)')
RPM_ESC_3=time_freq_feat('RPM(ESC_3)')
CURR_ESC_3=time_freq_feat('Curr(ESC_3)')
RPM_ESC_4=time_freq_feat('RPM(ESC_4)')
CURR_ESC_4=time_freq_feat('Curr(ESC_4)')
RPM_ESC_5=time_freq_feat('RPM(ESC_5)')
CURR_ESC_5=time_freq_feat('Curr(ESC_5)')
result = pd.merge([GyrX,GyrY,GyrZ,AccX,AccY,AccZ,Roll,Pitch,Yaw,RPM_ESC_0,CURR_ESC_0,RPM_ESC_1,CURR_ESC_1,RPM_ESC_2,CURR_ESC_2,RPM_ESC_3,CURR_ESC_3,RPM_ESC_4,CURR_ESC_4,RPM_ESC_5,CURR_ESC_5],axis=1)
display(result)


c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\chiar\AppData\Local\Temp\ipykernel_32680\342596788.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\chiar\AppData\Local\Temp\ipykernel_32680\342596788.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards th

,Ts(GyrX),<>(GyrX),var(GyrX),kurt(GyrX),RMS(GyrX),freq1(GyrX),freq2(GyrX),amp1(GyrX),amp2(GyrX)
0,0.0,-0.034370,0.019761,-0.785386,0.144520,73.0,71.0,4.020589,3.884841
1,1.0,-0.034917,0.019810,-0.801817,0.144820,88.0,86.0,4.186359,3.919004
2,2.0,-0.035273,0.019798,-0.809612,0.144867,77.0,75.0,4.410678,3.995558
3,3.0,-0.035721,0.019813,-0.821879,0.145028,76.0,74.0,4.594017,4.014788
4,4.0,-0.036275,0.019866,-0.837839,0.145349,83.0,81.0,4.730507,3.971330
5,5.0,-0.036635,0.019856,-0.845555,0.145405,72.0,70.0,4.927096,3.972665
6,6.0,-0.036972,0.019841,-0.852106,0.145439,71.0,69.0,5.122334,3.957618
7,7.0,-0.037434,0.019862,-0.864493,0.145629,70.0,68.0,5.263255,3.873946
8,8.0,-0.037917,0.019890,-0.877522,0.145852,79.0,77.0,5.386878,3.759917
9,9.0,-0.038191,0.019862,-0.880839,0.145827,76.0,75.0,5.575686,3.699033


c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\chiar\AppData\Local\Temp\ipykernel_32680\342596788.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\chiar\AppData\Local\Temp\ipykernel_32680\342596788.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards th

,Ts(GyrY),<>(GyrY),var(GyrY),kurt(GyrY),RMS(GyrY),freq1(GyrY),freq2(GyrY),amp1(GyrY),amp2(GyrY)
0,0.0,0.018781,0.002945,0.069954,0.057349,92.0,89.0,2.231465,2.145955
1,1.0,0.018713,0.002938,0.076029,0.057268,97.0,94.0,2.311873,2.005273
2,2.0,0.018698,0.002930,0.084744,0.057191,92.0,76.0,2.404229,1.883211
3,3.0,0.018635,0.002923,0.091154,0.057111,91.0,74.0,2.473536,1.911352
4,4.0,0.018547,0.002917,0.095184,0.057034,98.0,80.0,2.527760,1.753637
5,5.0,0.018515,0.002909,0.103429,0.056955,107.0,88.0,2.595321,1.574173
6,6.0,0.018347,0.002911,0.096473,0.056917,96.0,78.0,2.608511,1.610882
7,7.0,0.018123,0.002921,0.082752,0.056931,83.0,65.0,2.595798,1.808632
8,8.0,0.018071,0.002914,0.089852,0.056852,98.0,81.0,2.638854,1.993650
9,9.0,0.017866,0.002921,0.078155,0.056848,95.0,78.0,2.621926,1.869651


c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\chiar\AppData\Local\Temp\ipykernel_32680\342596788.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\chiar\AppData\Local\Temp\ipykernel_32680\342596788.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards th

...

c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\chiar\AppData\Local\Temp\ipykernel_32680\342596788.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\chiar\AppData\Local\Temp\ipykernel_32680\342596788.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards th

,Ts(RPM(ESC_1)),<>(RPM(ESC_1)),var(RPM(ESC_1)),kurt(RPM(ESC_1)),RMS(RPM(ESC_1)),freq1(RPM(ESC_1)),freq2(RPM(ESC_1)),amp1(RPM(ESC_1)),amp2(RPM(ESC_1))
0,0.0,4901.769231,678183.132308,-1.181716,4970.270876,40.0,41.0,8110.205792,5578.753516
1,1.0,4902.684659,676545.965804,-1.175442,4971.010041,54.0,55.0,7413.436797,5732.084967
2,2.0,4903.594901,674916.434860,-1.169157,4971.744909,56.0,57.0,6967.474873,5889.910200
3,3.0,4904.500000,673294.488669,-1.162861,4972.475518,48.0,49.0,7212.601655,6051.742737
4,4.0,4905.400000,671680.076836,-1.156555,4973.201904,54.0,55.0,7425.523379,6217.092353
5,5.0,4906.294944,670073.149383,-1.150238,4973.924104,56.0,57.0,7611.956933,6385.466484
6,6.0,4907.184874,668473.656737,-1.143910,4974.642155,52.0,53.0,7777.581143,6556.371588
7,7.0,4907.949721,666810.608109,-1.137551,4975.230494,46.0,47.0,7885.437257,6686.374046
8,8.0,4908.710306,665155.686793,-1.131185,4975.815487,54.0,55.0,7984.778151,6818.099603
9,9.0,4909.466667,663508.834540,-1.124811,4976.397161,48.0,49.0,8081.390445,6951.173713


c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\chiar\AppData\Local\Temp\ipykernel_32680\342596788.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\chiar\AppData\Local\Temp\ipykernel_32680\342596788.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards th

,Ts(Curr(ESC_1)),<>(Curr(ESC_1)),var(Curr(ESC_1)),kurt(Curr(ESC_1)),RMS(Curr(ESC_1)),freq1(Curr(ESC_1)),freq2(Curr(ESC_1)),amp1(Curr(ESC_1)),amp2(Curr(ESC_1))
0,0.0,2.622621,0.320503,-1.289886,2.682859,83.0,82.0,33.137817,11.404378
1,1.0,2.621733,0.319868,-1.286447,2.681873,73.0,72.0,34.061012,12.032287
2,2.0,2.620850,0.319235,-1.282980,2.680892,91.0,90.0,34.964344,12.594794
3,3.0,2.619972,0.318603,-1.279487,2.679917,81.0,80.0,35.847301,13.082932
4,4.0,2.619099,0.317974,-1.275966,2.678947,93.0,92.0,36.709396,13.488496
5,5.0,2.618230,0.317346,-1.272419,2.677982,91.0,90.0,37.550173,13.804141
6,6.0,2.617367,0.316721,-1.268845,2.677021,91.0,90.0,38.369202,14.023463
7,7.0,2.616145,0.316368,-1.266890,2.675762,95.0,94.0,39.036159,14.012352
8,8.0,2.614930,0.316014,-1.264885,2.674508,95.0,94.0,39.680825,13.898991
9,9.0,2.613722,0.315660,-1.262831,2.673261,93.0,92.0,40.303004,13.682551


KeyError: 'RPM(ESC_2)'

In [ ]:
GyrY=time_freq_feat('GyrY')